In [ ]:
# Note: this notebook is based on https://github.com/asigalov61/SuperPiano/blob/master/Super_Piano_3.ipynb
# except with NO training but using a pre-trained MT on Maestro dataset, as trained
# by Kern et al. (https://elifesciences.org/articles/80935)

# check if GPU is available (go to Edit > Notebook Settings > GPU if it's not)
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Thu Jun  1 14:28:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |       

In [ ]:
# install dependencies, including the pytorch MusicTransformer repos
# note: not sure these are all needed; I'm copying from
!git clone https://github.com/asigalov61/midi-neural-processor
!git clone https://github.com/asigalov61/MusicTransformer-Pytorch
!pip install tqdm
!pip install progress
!pip install pretty-midi
!pip install pypianoroll
!pip install matplotlib
!pip install librosa
!pip install scipy
!pip install pillow
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install mir_eval
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 /content/font.sf2

Cloning into 'midi-neural-processor'...
remote: Enumerating objects: 26, done.
Unpacking objects: 100% (26/26), 7.99 KiB | 1.33 MiB/s, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Cloning into 'MusicTransformer-Pytorch'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 385 (delta 99), reused 84 (delta 84), pack-reused 247
Receiving objects: 100% (385/385), 100.31 KiB | 2.05 MiB/s, done.
Resolving deltas: 100% (200/200), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9611 sha256=4ad232ec994653ecdd377f6213ef4a47ea6c591229aa813326c4ecc0ee13fa05
  Stored in directory: /root/.cache/pip/wheels/a2/68

In [ ]:
# move one clone repo into a subfolder (needed for some ugly path juggling in the MT repo...)
!mv midi-neural-processor/ MusicTransformer-Pytorch/third_party/midi_processor/

In [ ]:
# add the repo to the notebook path so we can use the code directly here
# (in the original notebook the notebook code then does system calls to pre-written python scripts,
# but this is considerably more instructive/useful)
import sys
sys.path.append('/content/MusicTransformer-Pytorch')

In [ ]:
# download weights
!cd /content/
!wget https://eelkespaak.nl/best_acc_weights.pickle

--2023-06-01 14:29:58--  https://eelkespaak.nl/best_acc_weights.pickle
Resolving eelkespaak.nl (eelkespaak.nl)... 185.104.29.80, 2a06:2ec0:1::118
Connecting to eelkespaak.nl (eelkespaak.nl)|185.104.29.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59429088 (57M)
Saving to: ‘best_acc_weights.pickle’

best_acc_weights.pi 100%[===================>]  56.68M  13.2MB/s    in 5.4s    

2023-06-01 14:30:04 (10.4 MB/s) - ‘best_acc_weights.pickle’ saved [59429088/59429088]



In [ ]:
import torch
import torch.nn as nn
import os

from third_party.midi_processor.processor import decode_midi, encode_midi
from model.music_transformer import MusicTransformer
from dataset.e_piano import process_midi

from utilities.constants import *
from utilities.device import get_device, use_cuda

# make sure we import numpy
import numpy as np

model_weights = '/content/best_acc_weights.pickle'
primer_file = '/content/output/ptrd.mid'
num_prime = 78
target_seq_length = 1200
rpr = True
output_dir = '/content/output'

# default values for architecture
# see utilities/argument_funcs.py for definitions etc.
n_layers = 6
num_heads = 8
d_model = 512
dim_feedforward = 1024
max_sequence = 2048

raw_mid = encode_midi(primer_file)
raw_mid_tensor = torch.tensor(raw_mid, dtype=torch.long)
primer, _ = process_midi(raw_mid_tensor, num_prime, random_seq=False)
primer = torch.tensor(primer, dtype=TORCH_LABEL_TYPE, device=get_device())

model = MusicTransformer(n_layers=n_layers, num_heads=num_heads,
            d_model=d_model, dim_feedforward=dim_feedforward,
            max_sequence=max_sequence, rpr=rpr).to(get_device())

model.load_state_dict(torch.load(model_weights))


# Saving primer first
f_path = os.path.join(output_dir, "primer.mid")
decode_midi(primer[:num_prime].cpu().numpy(), file_path=f_path)

## random sampling generation

model.eval()
with torch.set_grad_enabled(False):
    rand_seq = model.generate(primer[:num_prime], target_seq_length, beam=0)

## save to disk

f_path = os.path.join(output_dir, "rand.mid")
decode_midi(rand_seq[0].cpu().numpy(), file_path=f_path)

<ipython-input-78-6798793dc890>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  primer = torch.tensor(primer, dtype=TORCH_LABEL_TYPE, device=get_device())


Generating sequence of max length: 1200
100 / 1200
150 / 1200
200 / 1200
250 / 1200
300 / 1200
350 / 1200
400 / 1200
450 / 1200
500 / 1200
550 / 1200
600 / 1200
650 / 1200
700 / 1200
750 / 1200
800 / 1200
850 / 1200
900 / 1200
950 / 1200
1000 / 1200
1050 / 1200
1100 / 1200
1150 / 1200
1200 / 1200


In [ ]:
primer[:100]

tensor([381,  79, 267, 207, 268, 381,  79, 267, 207, 256, 381,  78, 267, 206,
        256, 381,  79, 267, 207, 268, 381,  79, 267, 207, 268, 381,  76, 267,
        204, 268, 381,  74, 267, 202, 268, 381,  72, 279, 200, 256, 381,  76,
        267, 204, 268, 381,  74, 267, 202, 268, 381,  74, 267, 202, 256, 381,
         73, 267, 201, 256, 381,  74, 267, 202, 268, 381,  74, 267, 202, 268,
        381,  71, 267, 199, 268, 381,  69, 267, 197, 389], device='cuda:0')